# Code for extracting thermal conductivity and viscosity
## with 9 machine learning algorithms


## Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.image as mpimg
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

## Data Preprocessing

In [ ]:
# Data for Nitrogen
# For other elements, choose the respective excel sheets

data1  = pd.DataFrame(pd.read_excel('DATASET_FINAL_MDFIX.xlsx', header=0, sheet_name='N_Visc'))
data2 = pd.DataFrame(pd.read_excel('DATASET_FINAL_MDFIX.xlsx', header=0, sheet_name='N_TC'))

X1 = pd.DataFrame(data1, columns=['T', 'P'])
y1 = data["V"]

X2 = pd.DataFrame(data2, columns=['T', 'P'])
y2 = data2["TC"]

list1=[X1,X2]

scaler = StandardScaler()
for i in list1:
    scaler.fit(i)
    p = scaler.transform(i)
    i = pd.DataFrame(p, columns=['T','P'])

datasets = [X1, X2]
targets = [y1, y2]

for i in range(2):
    X_train, X_test, y_train, y_test = train_test_split(datasets[i], targets[i], train_size=0.75, 
                                                       test_size=0.25, random_state=100)
    exec(f'X{i+1}_train = X_train')
    exec(f'X{i+1}_test = X_test')
    exec(f'y{i+1}_train = y_train')
    exec(f'y{i+1}_test = y_test')

Elements = ["Nitrogen Viscosity", "Nitrogen Thermal Conductivity"]

# Models

## AdaBoost Regressor

In [ ]:
models = []
mae_list = []
mse_list = []
r2_list = []

for i in range(2):
    model = AdaBoostRegressor(DecisionTreeRegressor(ccp_alpha=1.0, criterion='friedman_mse', max_depth=50, max_features='sqrt', min_samples_leaf=3, random_state=4), n_estimators=500, random_state=2, learning_rate=0.01)
    model.fit(eval(f'X{i+1}_train'), eval(f'y{i+1}_train'))
    ypred_train = model.predict(eval(f'X{i+1}_train'))
    ypred_test = model.predict(eval(f'X{i+1}_test'))
    mae = mean_absolute_error(eval(f'y{i+1}_test'), ypred_test)
    mse = mean_squared_error(eval(f'y{i+1}_test'), ypred_test)
    r2 = model.score(eval(f'X{i+1}_test'), eval(f'y{i+1}_test'))
    print(i,Elements[i-1])    
    models.append(model)
    mae_list.append(mae)
    mse_list.append(mse)
    r2_list.append(r2)

    exec(f'ada_ypred_train{i+1} = ypred_train')
    exec(f'ada_ypred_test{i+1} = ypred_test')
    exec(f'ada_mae{i+1} = mae')
    exec(f'ada_mse{i+1} = mse')
    exec(f'ada_R2{i+1} = r2')

## Gradient Boosting Regressor

In [ ]:
GBR_list = []
GBR_mae_list = []
GBR_mse_list = []
GBR_r2_list = []

for i in range(2):
    GBR = GradientBoostingRegressor(loss='squared_error', learning_rate=0.01, n_estimators=1350, 
                                    subsample=0.95, min_samples_split=5, max_depth=4, random_state=2)
    GBR.fit(eval(f'X{i+1}_train'), eval(f'y{i+1}_train'))
    ypred_train = GBR.predict(eval(f'X{i+1}_train'))
    ypred_test = GBR.predict(eval(f'X{i+1}_test'))
    mae = mean_absolute_error(eval(f'y{i+1}_test'), ypred_test)
    mse = mean_squared_error(eval(f'y{i+1}_test'), ypred_test)
    r2 = GBR.score(eval(f'X{i+1}_test'), eval(f'y{i+1}_test'))
    print(i,Elements[i-1])
    GBR_list.append(GBR)
    GBR_mae_list.append(mae)
    GBR_mse_list.append(mse)
    GBR_r2_list.append(r2)

    exec(f'GBR_ypred_train{i+1} = ypred_train')
    exec(f'GBR_ypred_test{i+1} = ypred_test')
    exec(f'GBR_mae{i+1} = mae')
    exec(f'GBR_mse{i+1} = mse')
    exec(f'GBR_R2{i+1} = r2')

## Gaussian Process Regression

In [ ]:
GPR_list = []
GPR_mae_list = []
GPR_mse_list = []
GPR_r2_list = []

kernel = Matern(length_scale=1.0, nu=0.7)

models = []

for i in range(1,3):
     Ada = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=100)
     Ada.fit(eval(f'X{i}_train'), eval(f'y{i}_train'))
     ypred_train = Ada.predict(eval(f'X{i}_train'))
     ypred_test = Ada.predict(eval(f'X{i}_test'))
     mae = mean_absolute_error(eval(f'y{i}_test'), ypred_test)
     mse = mean_squared_error(eval(f'y{i}_test'), ypred_test)
     r2 = Ada.score(eval(f'X{i}_test'), eval(f'y{i}_test'))
     print(i,Elements[i-1])
     GPR_list.append(Ada)
     GPR_mae_list.append(mae)
     GPR_mse_list.append(mse)
     GPR_r2_list.append(r2)

     exec(f'GPR_ypred_train{i} = ypred_train')
     exec(f'GPR_ypred_test{i} = ypred_test')
     exec(f'GPR_mae{i} = mae')
     exec(f'GPR_mse{i} = mse')
     exec(f'GPR_R2{i} = r2')
    
#     model_filename = f'GPR_model_{i}.joblib'
#     dump(Ada, model_filename)

## Multi-layer Perceptron

In [ ]:
MLP_list = []
MLP_mae_list = []
MLP_mse_list = []
MLP_r2_list = []

for i in range(1,3):
        MLP = MLPRegressor(hidden_layer_sizes=(10, 10), random_state=100, max_iter=7000)
        MLP.fit(eval(f'X{i}_train'), eval(f'y{i}_train'))
        ypred_train = MLP.predict(eval(f'X{i}_train'))
        ypred_test = MLP.predict(eval(f'X{i}_test'))
        mae = mean_absolute_error(eval(f'y{i}_test'), ypred_test)
        mse = mean_squared_error(eval(f'y{i}_test'), ypred_test)
        r2 = MLP.score(eval(f'X{i}_test'), eval(f'y{i}_test'))

        print(i,Elements[i-1])

        MLP_list.append(MLP)
        MLP_mae_list.append(mae)
        MLP_mse_list.append(mse)
        MLP_r2_list.append(r2)

        exec(f'MLP_ypred_train{i} = ypred_train')
        exec(f'MLP_ypred_test{i} = ypred_test')
        exec(f'MLP_mae{i} = mae')
        exec(f'MLP_mse{i} = mse')
        exec(f'MLP_R2{i} = r2')


## k-nearest neighbors

In [ ]:
k_range = range(1, 21)

KNN_list = []
KNN_mae_list = []
KNN_mse_list = []
KNN_r2_list = []

for i in range(1,3):
        KNN = KNeighborsRegressor(n_neighbors=3, weights='distance', algorithm='auto', leaf_size=1, p=1, metric='minkowski')
        KNN.fit(eval(f'X{i}_train'), eval(f'y{i}_train'))
        ypred_train = KNN.predict(eval(f'X{i}_train'))
        ypred_test = KNN.predict(eval(f'X{i}_test'))
        mae = mean_absolute_error(eval(f'y{i}_test'), ypred_test)
        mse = mean_squared_error(eval(f'y{i}_test'), ypred_test)
        r2 = KNN.score(eval(f'X{i}_test'), eval(f'y{i}_test'))

        print(i,Elements[i-1])

        KNN_list.append(KNN)
        KNN_mae_list.append(mae)
        KNN_mse_list.append(mse)
        KNN_r2_list.append(r2)

        exec(f'KNN_ypred_train{i} = ypred_train')
        exec(f'KNN_ypred_test{i} = ypred_test')
        exec(f'KNN_mae{i} = mae')
        exec(f'KNN_mse{i} = mse')
        exec(f'KNN_R2{i} = r2')

## Random Forest

In [ ]:
RFR_list = []
RFR_mae_list = []
RFR_mse_list = []
RFR_r2_list = []

for i in range(1,3):
    RFR = RandomForestRegressor(n_estimators=1000, criterion='squared_error', max_depth=50, min_samples_split=5,  min_samples_leaf=1, max_features='sqrt', random_state=2)
    RFR.fit(eval(f'X{i}_train'), eval(f'y{i}_train'))
    ypred_train = RFR.predict(eval(f'X{i}_train'))
    ypred_test = RFR.predict(eval(f'X{i}_test'))
    mae = mean_absolute_error(eval(f'y{i}_test'), ypred_test)
    mse = mean_squared_error(eval(f'y{i}_test'), ypred_test)
    r2 = RFR.score(eval(f'X{i}_test'), eval(f'y{i}_test'))

    print(i,Elements[i-1])

    RFR_list.append(RFR)
    RFR_mae_list.append(mae)
    RFR_mse_list.append(mse)
    RFR_r2_list.append(r2)

    exec(f'RFR_ypred_train{i} = ypred_train')
    exec(f'RFR_ypred_test{i} = ypred_test')
    exec(f'RFR_mae{i} = mae')
    exec(f'RFR_mse{i} = mse')
    exec(f'RFR_R2{i} = r2')
    

## Support Vector Regression

In [ ]:
from sklearn.svm import SVR
SVR_list = []
SVR_mae_list = []
SVR_mse_list = []
SVR_r2_list = []

params = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'epsilon': [0.1, 0.01, 0.001, 0.0001]}

for i in range(1,3):
        model = SVR(kernel='rbf')
        grid_search = GridSearchCV(model, params, cv=5, n_jobs=-1)
        grid_search.fit(eval(f'X{i}_train'), eval(f'y{i}_train'))
        model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        best_model = grid_search.best_estimator_

        model.fit(eval(f'X{i}_train'), eval(f'y{i}_train'))
        ypred_train = model.predict(eval(f'X{i}_train'))
        ypred_test = model.predict(eval(f'X{i}_test'))
        mae = mean_absolute_error(eval(f'y{i}_test'), ypred_test)
        mse = mean_squared_error(eval(f'y{i}_test'), ypred_test)
        r2 = r2_score(eval(f'y{i}_test'), ypred_test)
        print(f'{i} : {Elements[i-1]}')
        print("Best Hyperparameters:", best_params)
        print("R^2 Score on Test Set:", r2)
        SVR_list.append(model)
        SVR_mae_list.append(mae)
        SVR_mse_list.append(mse)
        SVR_r2_list.append(r2)

        exec(f'SVR_ypred_train{i} = ypred_train')
        exec(f'SVR_ypred_test{i} = ypred_test')
        exec(f'SVR_mae{i} = mae')
        exec(f'SVR_mse{i} = mse')
        exec(f'SVR_R2{i} = r2')

## Extra Trees Regressor

In [ ]:
ETR_list = []
ETR_mae_list = []
ETR_mse_list = []
ETR_r2_list = []

for i in range(1,3):
    ETR = ExtraTreesRegressor(n_estimators=1000, criterion='squared_error', max_depth=50, min_samples_split=5,  min_samples_leaf=1, max_features='sqrt', random_state=2)
    ETR.fit(eval(f'X{i}_train'), eval(f'y{i}_train'))

    ypred_train = ETR.predict(eval(f'X{i}_train'))
    ypred_test = ETR.predict(eval(f'X{i}_test'))
    mae = mean_absolute_error(eval(f'y{i}_test'), ypred_test)
    mse = mean_squared_error(eval(f'y{i}_test'), ypred_test)
    r2 = ETR.score(eval(f'X{i}_test'), eval(f'y{i}_test'))
    print(i,Elements[i-1])
    ETR_list.append(ETR)
    ETR_mae_list.append(mae)
    ETR_mse_list.append(mse)
    ETR_r2_list.append(r2)

    exec(f'ETR_ypred_train{i} = ypred_train')
    exec(f'ETR_ypred_test{i} = ypred_test')
    exec(f'ETR_mae{i} = mae')
    exec(f'ETR_mse{i} = mse')
    exec(f'ETR_R2{i} = r2')

## Decision Tree Regressor

In [ ]:
DTR_list = []
DTR_mae_list = []
DTR_mse_list = []
DTR_r2_list = []

DTR = DecisionTreeRegressor(ccp_alpha=1.0, criterion='friedman_mse', max_depth=50, max_features='sqrt', min_samples_leaf=3, random_state=4, splitter='best')
for i in range(1,3):
    DTR.fit(eval(f'X{i}_train'), eval(f'y{i}_train'))

    ypred_train = DTR.predict(eval(f'X{i}_train'))
    ypred_test = DTR.predict(eval(f'X{i}_test'))
    mae = mean_absolute_error(eval(f'y{i}_test'), ypred_test)
    mse = mean_squared_error(eval(f'y{i}_test'), ypred_test)
    r2 = DTR.score(eval(f'X{i}_test'), eval(f'y{i}_test'))
    print(i,Elements[i-1])
    DTR_list.append(DTR)
    DTR_mae_list.append(mae)
    DTR_mse_list.append(mse)
    DTR_r2_list.append(r2)

    exec(f'DTR_ypred_train{i} = ypred_train')
    exec(f'DTR_ypred_test{i} = ypred_test')
    exec(f'DTR_mae{i} = mae')
    exec(f'DTR_mse{i} = mse')
    exec(f'DTR_R2{i} = r2')

# Graphs

In [ ]:
algorithm_names = ['a) AdaBoost Regression', 'b) Gradient Boosting Regression', 'c) Gaussian Process Regression', 'd) Multi-Layer Perceptron', 'e) k-nearest neighbors', 'f) Random Forest Regression', 'g) Support Vector Regression', 'h) Extra Trees Regression', 'i) Decision Tree Regression']

# Create the figure and axes objects
line_labels = ["test","train"]


def plotmyaxes(a1, a2, y_test, y_pred, y_train, y_predt,R2,number):
    

    l1 = axs[a1, a2].scatter(y_test, y_pred, c='r', marker='o', s=10, alpha=1)
    l2 = axs[a1, a2].scatter(y_train, y_predt, c='b', marker='^', s=10, alpha=0.5)
    axs[a1, a2].legend([l1, l2], labels=line_labels, loc="lower right",   # Position of legend
                borderaxespad=0.5, fontsize=12
               )
    # Plot the diagonal line
    p1 = max(max(y_train), max(y_test))
    axs[a1, a2].plot([0, p1], [0, p1], 'k-', linewidth=0.5)
    axs[a1, a2].set_xlim([0, None])
    axs[a1, a2].set_ylim([0, None])
    # Set the subplot title
    axs[a1, a2].set_title(algorithm_names[a1*3 + a2])
    # Turn on the grid
    axs[a1, a2].grid(True)

    

    if number in [0,2,4,6,9]:
        x_title = r'$\eta_{real}$'
        y_title = r'$\eta_{pred}$'
        for ax in axs.flat:
            ax.set(xlabel=x_title, ylabel=y_title)
    else:
        x_title = r'$\lambda_{real}$'
        y_title = r'$\lambda_{pred}$'
        for ax in axs.flat:
            ax.set(xlabel=x_title, ylabel=y_title)

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:

        ax.label_outer()
    
    print(R2)



## Nitrogen Viscosity

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(15, 12))
plotmyaxes(0,0,y1_test,ada_ypred_test1, y1_train, ada_ypred_train1,ada_R21,0)
plotmyaxes(0,1,y1_test,GBR_ypred_test1, y1_train, GBR_ypred_train1,GBR_R21,0)
plotmyaxes(0,2,y1_test,GPR_ypred_test1, y1_train, GPR_ypred_train1,GPR_R21,0)
plotmyaxes(1,0,y1_test,MLP_ypred_test1, y1_train, MLP_ypred_train1,MLP_R21,0)
plotmyaxes(1,1,y1_test,KNN_ypred_test1, y1_train, KNN_ypred_train1,KNN_R21,0)
plotmyaxes(1,2,y1_test,RFR_ypred_test1, y1_train, RFR_ypred_train1,RFR_R21,0)
plotmyaxes(2,0,y1_test,SVR_ypred_test1, y1_train, SVR_ypred_train1,SVR_R21,0)
plotmyaxes(2,1,y1_test,ETR_ypred_test1, y1_train, ETR_ypred_train1,ETR_R21,0)
plotmyaxes(2,2,y1_test,DTR_ypred_test1, y1_train, DTR_ypred_train1,DTR_R21,0)
print(Elements[0])
fig.suptitle(Elements[0], fontsize=30)
fig.tight_layout(pad=3.0)
fig.show()

## Nitrogen Thermal Conductivity

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(15, 12))
plotmyaxes(0,0,y2_test, ada_ypred_test2, y2_train, ada_ypred_train2,ada_R22,1)
plotmyaxes(0,1,y2_test, GBR_ypred_test2, y2_train, GBR_ypred_train2,GBR_R22,1)
plotmyaxes(0,2,y2_test, GPR_ypred_test2, y2_train, GPR_ypred_train2,GPR_R22,1)
plotmyaxes(1,0,y2_test, MLP_ypred_test2, y2_train, MLP_ypred_train2,MLP_R22,1)
plotmyaxes(1,1,y2_test, KNN_ypred_test2, y2_train, KNN_ypred_train2,KNN_R22,1)
plotmyaxes(1,2,y2_test, RFR_ypred_test2, y2_train, RFR_ypred_train2,RFR_R22,1)
plotmyaxes(2,0,y2_test, SVR_ypred_test2, y2_train, SVR_ypred_train2,SVR_R22,1)
plotmyaxes(2,1,y2_test, ETR_ypred_test2, y2_train, ETR_ypred_train2,ETR_R22,1)
plotmyaxes(2,2,y2_test, DTR_ypred_test2, y2_train, DTR_ypred_train2,DTR_R22,1)
print(Elements[1])
fig.suptitle(Elements[1], fontsize=30)
fig.tight_layout(pad=3.0)
fig.show()

### The same procedure can be followed for the remaining elements (Ar, Kr, Xe, O)